# Description

# Modules and Global Variables

In [ ]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, 
    TrainingArguments, Trainer,
)

import torch
import wandb

import datasets
import evaluate

import numpy as np
import pandas as pd

import os
import re
import random

In [ ]:
print(f'torch.__version__: {torch.__version__}')
print(f'torch.cuda.is_available(): {torch.cuda.is_available()}')
NGPU = torch.cuda.device_count()
print(f'NGPU: {NGPU}')

In [ ]:
### labels

ce_labels = ['True', 'False']
pc_labels = ['positive', 'negative', 'neutral']
pc_binary_labels = ['True', 'False']

labels = pc_labels

label2id = {k: i for i, k in enumerate(labels)}
id2label = {i: k for i, k in enumerate(labels)}
num_labels = len(labels)

print(label2id)
print(id2label)

In [ ]:
### paths and names

PROJECT_NAME = 'aspect_sentiment_classification_multi'
RUN_ID = 'uncleaned_v13_maxlen_256'

DATA_V = 'uncleaned_v13'
DATA_T = 'pc' # ce or pc
AUGMENTATION = False
AUG_NAME = 'balanced'

model_checkpoint = 'klue_roberta_base_mlm_fine_tuned'

notebook_name = 'trainer_for_asc_m_new.ipynb'

### fixed

model_name = re.sub(r'[/-]', r'_', model_checkpoint).lower()
run_name = f'{model_name}_{RUN_ID}'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, 'training_results', RUN_ID, 'asc')
NOTEBOOK_PATH = os.path.join(ROOT_PATH, notebook_name)

augornot = f'_{AUG_NAME}' if AUGMENTATION is True else ''
TRAIN_DATA_PATH = os.path.join(ROOT_PATH, 'dataset', DATA_V, f'{DATA_T}_train{augornot}.csv')
EVAL_DATA_PATH = os.path.join(ROOT_PATH, 'dataset', DATA_V, f'{DATA_T}_dev.csv')

!mkdir -p {SAVE_PATH}

In [ ]:
print(f'model_checkpoint: {model_checkpoint}')
print(f'DATA_V: {DATA_V}')
if os.path.exists(SAVE_PATH):
    print(f'{SAVE_PATH} exists.')
else:
    print(f'{SAVE_PATH} does not exist.')
if os.path.exists(NOTEBOOK_PATH):
    print(f'{NOTEBOOK_PATH} exists.')
else:
    print(f'{NOTEBOOK_PATH} does not exist.')
if os.path.exists(TRAIN_DATA_PATH):
    print(f'{TRAIN_DATA_PATH} exists.')
else:
    print(f'{TRAIN_DATA_PATH} does not exist.')
if os.path.exists(EVAL_DATA_PATH):
    print(f'{EVAL_DATA_PATH} exists.')
else:
    print(f'{EVAL_DATA_PATH} does not exist.')

In [ ]:
### rest of training args

report_to="wandb"

fp16 = False

num_train_epochs = 10
batch_size = 8
gradient_accumulation_steps = 1

optim = 'adamw_hf' # 'adamw_torch'

learning_rate = 3e-6 / 8 * batch_size * NGPU # 5e-5
weight_decay = 0.01 # 0
adam_epsilon = 1e-8

lr_scheduler_type = 'linear'
warmup_ratio = 0

save_total_limit = 2

load_best_model_at_end = True
metric_for_best_model ='f1_macro'

save_strategy = "epoch"
evaluation_strategy = "epoch"

logging_strategy = "steps"
logging_first_step = True 
logging_steps = 100

print(f'batch_size: {batch_size}')
print(f'learning_rate: {learning_rate}')
print(f'num_train_epochs: {num_train_epochs}')

# WandB Configuration

In [ ]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

# Load Model, Tokenizer, and Collator

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, label2id=label2id, id2label=id2label, num_labels=num_labels
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length', max_length=256)

In [ ]:
# train_path = f'./dataset/{DATA_V}/raw_train.csv'
# dev_path = f'./dataset/{DATA_V}/raw_dev.csv'
# test_path = f'./dataset/{DATA_V}/raw_test.csv'
# train = pd.read_csv(train_path)
# dev = pd.read_csv(dev_path)
# test = pd.read_csv(test_path)

# print(len(tokenizer))
# tokenizer_train_data = pd.concat([train.sentence_form, dev.sentence_form, test.sentence_form]).to_frame().drop_duplicates()
# tokenizer_train_data = tokenizer_train_data.sentence_form.to_list()
# new_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# new_tokenizer = tokenizer.train_new_from_iterator(tokenizer_train_data, vocab_size=1)
# new_tokens = set(list(new_tokenizer.vocab.keys())) - set(tokenizer.vocab.keys())
# tokenizer.add_tokens(list(new_tokens))
# print(len(new_tokenizer))
# print(len(tokenizer))
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
# print(len(new_tokens))
# print(new_tokens)

In [ ]:
model.config.label2id, model.config.id2label, model.num_labels

# Define Metric

In [ ]:
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_metric.compute(references=labels, predictions=predictions)['accuracy']
    f1_positive, f1_negative, f1_neutral = tuple(f1_metric.compute(references=labels, predictions=predictions, average=None, labels=[0,1,2])['f1'])
    f1_macro = f1_metric.compute(references=labels, predictions=predictions, average='macro')['f1']
    f1_micro = f1_metric.compute(references=labels, predictions=predictions, average='micro')['f1']
    
    return {'accuracy': accuracy, 
            'f1_positive': f1_positive, 'f1_negative': f1_negative, 'f1_neutral': f1_neutral, 
            'f1_macro': f1_macro, 'f1_micro': f1_micro}

# Load Data

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["form"], examples["pair"], truncation=True)

In [ ]:
train_dataset = pd.read_csv(TRAIN_DATA_PATH)
eval_dataset = pd.read_csv(EVAL_DATA_PATH)
train_dataset = datasets.Dataset.from_pandas(train_dataset).shuffle(seed=42)
eval_dataset = datasets.Dataset.from_pandas(eval_dataset).shuffle(seed=42)
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

In [ ]:
len(train_dataset), len(eval_dataset)

In [ ]:
k = random.randrange(len(train_dataset))
print(tokenizer.decode(train_dataset['input_ids'][k]), train_dataset['labels'][k])
k = random.randrange(len(eval_dataset))
print(tokenizer.decode(eval_dataset['input_ids'][k]), eval_dataset['labels'][k])

# Load Trainer

In [ ]:
args = TrainingArguments(
    output_dir=run_name,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,
    
    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,
    
    fp16=fp16,
)

In [ ]:
# es = EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    # callbacks=[es],
)

# Run Trainer

In [ ]:
trainer.train()
wandb.finish()

In [ ]:
keep = [
    'added_tokens.json',
    'config.json',
    'pytorch_model.bin',
    'special_tokens_map.json',
    'tokenizer.json',
    'tokenizer_config.json',
    'vocab.txt'
]

ckpts = os.listdir(run_name)
for ckpt in ckpts:
    ckpt = os.path.join(run_name, ckpt)
    for item in os.listdir(ckpt):
        if item not in keep:
            os.remove(os.path.join(ckpt, item))

!mv wandb {run_name} {SAVE_PATH}/